In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
x_train = x_train.reshape(-1, 784) / 255.0  
x_test = x_test.reshape(-1, 784) / 255.0

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [6]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/5
938/938 [==============================] - 5s 4ms/step - loss: 0.5194 - accuracy: 0.8135 - val_loss: 0.4109 - val_accuracy: 0.8544
Epoch 2/5
938/938 [==============================] - 3s 3ms/step - loss: 0.3722 - accuracy: 0.8643 - val_loss: 0.4197 - val_accuracy: 0.8409
Epoch 3/5
938/938 [==============================] - 3s 3ms/step - loss: 0.3349 - accuracy: 0.8767 - val_loss: 0.3755 - val_accuracy: 0.8660
Epoch 4/5
938/938 [==============================] - 3s 3ms/step - loss: 0.3118 - accuracy: 0.8835 - val_loss: 0.3450 - val_accuracy: 0.8739
Epoch 5/5
938/938 [==============================] - 4s 4ms/step - loss: 0.2923 - accuracy: 0.8915 - val_loss: 0.3361 - val_accuracy: 0.8780


In [8]:
def prune_weights(model, threshold):
    pruned_model = tf.keras.models.clone_model(model)
    pruned_model.set_weights(model.get_weights())  # Copy the original weights

    # Prune the weights below the threshold
    for layer in pruned_model.layers:
        if isinstance(layer, tf.keras.layers.Dense):
            weights = layer.get_weights()
            weights[0][weights[0] < threshold] = 0.0  # Prune weights below threshold
            layer.set_weights(weights)
    
    return pruned_model


In [15]:

thresholds = [ -0.1, -0.2, -0.3, -0.5, -0.7,-1,0.0, 0.1, 0.2, 0.3, 0.5, 0.7,1]
performance = []

In [16]:
for threshold in thresholds:
    pruned_model = prune_weights(model, threshold)
    pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    pruned_model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))
    loss, accuracy = pruned_model.evaluate(x_test, y_test)
    performance.append(accuracy)

Epoch 1/5
938/938 [==============================] - 5s 5ms/step - loss: 0.8407 - accuracy: 0.8330 - val_loss: 0.3765 - val_accuracy: 0.8635
Epoch 2/5
938/938 [==============================] - 4s 5ms/step - loss: 0.3190 - accuracy: 0.8841 - val_loss: 0.3714 - val_accuracy: 0.8686
Epoch 3/5
938/938 [==============================] - 4s 4ms/step - loss: 0.2967 - accuracy: 0.8908 - val_loss: 0.3700 - val_accuracy: 0.8729
Epoch 4/5
938/938 [==============================] - 4s 4ms/step - loss: 0.2816 - accuracy: 0.8969 - val_loss: 0.3438 - val_accuracy: 0.8782
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.3440 - accuracy: 0.8760
Epoch 1/5
938/938 [==============================] - 4s 4ms/step - loss: 0.3126 - accuracy: 0.8858 - val_loss: 0.3647 - val_accuracy: 0.8733
Epoch 2/5
938/938 [==============================] - 4s 4ms/step - loss: 0.2800 - accuracy: 0.8957 - val_loss: 0.3640 - val_accuracy: 0.8692
Epoch 3/5
938/938 [==============================] - 4s

In [17]:
acc_on_t = []

for threshold, acc in zip(thresholds, performance):
    ind = []
    ind.append(f'Threshold: {threshold}, Accuracy: {acc}')
    acc_on_t.append(ind)
print(acc_on_t)

[['Threshold: -0.1, Accuracy: 0.8759999871253967'], ['Threshold: -0.2, Accuracy: 0.8837000131607056'], ['Threshold: -0.3, Accuracy: 0.8835999965667725'], ['Threshold: -0.5, Accuracy: 0.8841000199317932'], ['Threshold: -0.7, Accuracy: 0.8823999762535095'], ['Threshold: -1, Accuracy: 0.8791999816894531'], ['Threshold: 0.0, Accuracy: 0.8671000003814697'], ['Threshold: 0.1, Accuracy: 0.8744999766349792'], ['Threshold: 0.2, Accuracy: 0.8741999864578247'], ['Threshold: 0.3, Accuracy: 0.8695999979972839'], ['Threshold: 0.5, Accuracy: 0.8460999727249146'], ['Threshold: 0.7, Accuracy: 0.8565999865531921'], ['Threshold: 1, Accuracy: 0.8478000164031982']]
